**39. Combination Sum**

Given an array of distinct integers candidates and a target integer target, return a list of all unique combinations of candidates where the chosen numbers sum to target. You may return the combinations in any order.

The same number may be chosen from candidates an unlimited number of times. Two combinations are unique if the frequency of at least one of the chosen numbers is different.

The test cases are generated such that the number of unique combinations that sum up to target is less than 150 combinations for the given input.

https://leetcode.com/problems/combination-sum/

In [78]:
# dynamic programming
def combinationSum(candidates: list[int], target: int) -> list[list[int]]:
    candidates.sort()
    dp = [[] for _ in range(target+1)]
    for t in range(candidates[0], target+1):
        for c in candidates:
            if c > t:
                break
            elif c == t:
                dp[c].append([c])
            else:
                if dp[t-c]:
                    for l in dp[t-c]:
                        # only appending values that are smaller or equal
                        # than the last value in the candidate solution.
                        # this ensures that solutions are unique
                        if c <= l[-1]:
                            dp[t].append(l + [c])
    return dp[target]

# bfs
def combinationSum(candidates: list[int], target: int) -> list[list[int]]:
    from collections import deque
    candidates.sort()
    ans = []
    q = deque([([], 0, target)])
    while q:
        solution, index, new_target = q.popleft()
        for i in range(index, len(candidates)):
            if candidates[i] > new_target:
                break
            if candidates[i] == new_target:
                ans.append(solution + [candidates[i]])
            else:
                q.append((solution + [candidates[i]], i, new_target-candidates[i]))
    return ans


print(combinationSum(candidates = [2,3,6,7], target = 7)) # [[2,2,3],[7]]
print(combinationSum(candidates = [2,3,5], target = 8)) # [[2,2,2,2],[2,3,3],[3,5]]
print(combinationSum(candidates = [2], target = 1)) # []
print(combinationSum(candidates = [2], target = 2)) # [[2]]

[[7], [2, 2, 3]]
[[3, 5], [2, 3, 3], [2, 2, 2, 2]]
[]
[[2]]


**40. Combination Sum II**

Given a collection of candidate numbers (candidates) and a target number (target), find all unique combinations in candidates where the candidate numbers sum to target.

Each number in candidates may only be used once in the combination.

Note: The solution set must not contain duplicate combinations.

https://leetcode.com/problems/combination-sum-ii/

In [56]:
# bottom-up DP
def combinationSum2(candidates: list[int], target: int) -> list[list[int]]:
    candidates.sort()
    dp = [{()}] + [set() for _ in range(target)]
    for c in candidates:
        for t in range(target, c-1, -1):
            for tup in dp[t-c]:
                dp[t].add(tup + (c,))
    return dp[target]

# BFS
from collections import deque
def combinationSum2(candidates: list[int], target: int) -> list[list[int]]:
    candidates.sort()
    ans = []
    q = deque([(0, [], target)])
    while q:
        index, cand, new_target = q.pop()
        if new_target == 0:
            ans.append(cand)
        else:
            for i in range(index, len(candidates)):
                if i > index and candidates[i] == candidates[i-1]:
                    continue
                if candidates[i] > new_target:
                    break
                q.appendleft((i+1, cand+[candidates[i]], new_target-candidates[i]))
    return ans

# recursive
def combinationSum2(candidates: list[int], target: int) -> list[list[int]]:
    candidates.sort()
    ans = []
    def backtrack(lst, ind, tgt):
        # basecase
        if tgt == 0:
            ans.append(lst)
            return
        for i in range(ind, len(candidates)):
            if i > ind and candidates[i] == candidates[i-1]:
                continue
            if candidates[i] > tgt:
                break
            backtrack(lst+[candidates[i]], i+1, tgt-candidates[i])

    backtrack([], 0, target)
    return ans

print(combinationSum2(candidates = [10,1,2,7,6,1,5], target = 8)) # [[1,1,6], [1,2,5], [1,7], [2,6]]
print(combinationSum2(candidates = [2,5,2,1,2], target = 5)) # [[1,2,2], [5]]
print(combinationSum2(candidates = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1], target = 27)) # []

[[1, 1, 6], [1, 2, 5], [1, 7], [2, 6]]
[[1, 2, 2], [5]]
[]


**45. Jump Game II**

You are given a 0-indexed array of integers nums of length n. You are initially positioned at nums[0].

Each element nums[i] represents the maximum length of a forward jump from index i. In other words, if you are at nums[i], you can jump to any nums[i + j] where:

* 0 <= j <= nums[i] and
* i + j < n

Return the minimum number of jumps to reach nums[n - 1]. The test cases are generated such that you can reach nums[n - 1].

In [2]:
def jump(nums: list[int]) -> int:
    if len(nums) > 1 and nums[0] >= len(nums)-1:
        return 1
    current = -1
    nxt = nums[0]
    jumps = 0
    for i in range(len(nums)-1):
        if i > current:
            current = nxt
            jumps += 1
        nxt = max(nxt, nums[i] + i)
        if nxt >= len(nums) - 1:
            return jumps + 1
    return jumps

**58. Length of Last Word**

Given a string s consisting of words and spaces, return the length of the last word in the string.

A word is a maximal substring consisting of non-space characters only.

In [ ]:
# one liner
def lengthOfLastWord(s: str) -> int:
    return len(s.rstrip().split(" ")[-1])

# another solution
def lengthOfLastWord(s: str) -> int:
    word = None
    for i in range(len(s)-1, -1, -1):
        if s[i] == " ":
            if word:
                return word - i
        elif not word:
            word = i
    return word + 1

**71. Simplify Path**

Given a string path, which is an absolute path (starting with a slash '/') to a file or directory in a Unix-style file system, convert it to the simplified canonical path.

In a Unix-style file system, a period '.' refers to the current directory, a double period '..' refers to the directory up a level, and any multiple consecutive slashes (i.e. '//') are treated as a single slash '/'. For this problem, any other format of periods such as '...' are treated as file/directory names.

The canonical path should have the following format:

* The path starts with a single slash '/'.
* Any two directories are separated by a single slash '/'.
* The path does not end with a trailing '/'.
* The path only contains the directories on the path from the root directory to the target file or directory (i.e., no period '.' or double period '..')

Return the simplified canonical path.

https://leetcode.com/problems/simplify-path/

In [74]:
def simplifyPath(path: str) -> str:
    path += "/"
    ans = []
    current = []
    dots = 0
    for c in path:
        if c == "/":
            if dots == len(current) == 2:
                if ans:
                    ans.pop()
            elif current and current != ["."]:
                ans.append(current)
            dots = 0
            current = []
        else:
            if c == ".":
                dots += 1
            current.append(c)
    return "/" + "/".join(["".join(word) for word in ans])
        

print(simplifyPath("/home/")) # "/home"
print(simplifyPath("/../")) # "/"
print(simplifyPath("/home//foo/")) # "/home/foo"
print(simplifyPath("/a//b////c/d//././/..")) # "/a/b/c"
print(simplifyPath("/..hidden")) # "/..hidden"
print(simplifyPath("/hello../world")) # "/hello../world"
print(simplifyPath("/...")) # "/..."
print(simplifyPath("/hello./world/")) # "/hello./world"
print(simplifyPath("/../..ga/b/.f..d/..../e.baaeeh./.a")) # "/..ga/b/.f..d/..../e.baaeeh./.a"

/home
/
/home/foo
/a/b/c
/..hidden
/hello../world
/...
/hello./world
/..ga/b/.f..d/..../e.baaeeh./.a


**83. Remove Duplicates from Sorted List**

Given the head of a sorted linked list, delete all duplicates such that each element appears only once. Return the linked list sorted as well.

In [5]:
def deleteDuplicates(head):
    if not head or not head.next:
        return head
    node = head 
    while node.next:
        if node.val == node.next.val:
            node.next = node.next.next
        else:
            node = node.next
    return head 

**216. Combination Sum III**

Find all valid combinations of k numbers that sum up to n such that the following conditions are true:

Only numbers 1 through 9 are used.
Each number is used at most once.
Return a list of all possible valid combinations. The list must not contain the same combination twice, and the combinations may be returned in any order.

https://leetcode.com/problems/combination-sum-iii/

In [73]:
# dfs
def combinationSum3(k: int, n: int) -> list[list[int]]:
    if n < k or n > (9 * k):
        return []
    ans = []
    s = [([], 1, n)]
    while s:
        lst, num, tgt = s.pop()
        if len(lst) == k:
            if tgt == 0:
                ans.append(lst)
        else:
            for i in range(num, 10):
                s.append((lst + [i], i+1, tgt-i))
    return ans

# recursive
def combinationSum3(k: int, n: int) -> list[list[int]]:
    if n < k or n > (9 * k):
        return []
    ans = []
    
    def dfs(lst, num, tgt):
        if len(lst) == k:
            if tgt == 0:
                ans.append(lst)
            return
        for i in range(num, 10):
            dfs(lst + [i], i+1, tgt-i)

    dfs([], 1, n)

    return ans

print(combinationSum3(k = 3, n = 7))
print(combinationSum3(k = 3, n = 9))
print(combinationSum3(k = 4, n = 1))

[[1, 2, 4]]
[[1, 2, 6], [1, 3, 5], [2, 3, 4]]
[]


**1857. Largest Color Value in a Directed Graph**

There is a directed graph of n colored nodes and m edges. The nodes are numbered from 0 to n - 1.

You are given a string colors where colors[i] is a lowercase English letter representing the color of the ith node in this graph (0-indexed). You are also given a 2D array edges where edges[j] = [aj, bj] indicates that there is a directed edge from node aj to node bj.

A valid path in the graph is a sequence of nodes x1 -> x2 -> x3 -> ... -> xk such that there is a directed edge from xi to xi+1 for every 1 <= i < k. The color value of the path is the number of nodes that are colored the most frequently occurring color along that path.

Return the largest color value of any valid path in the given graph, or -1 if the graph contains a cycle.

https://leetcode.com/problems/largest-color-value-in-a-directed-graph/

In [54]:
# times out
def largestPathValue(colors: str, edges: list[list[int]]) -> int:
    if not edges:
        return 1
    
    graph = {}
    from_nodes = set()
    to_nodes = set()
    for k, v in edges:
        from_nodes.add(k)
        to_nodes.add(v)
        if k in graph:
            graph[k].append(v)
        else:
            graph[k] = [v]

    from_nodes -= to_nodes

    if not from_nodes:
        return -1
    
    largest_color_value = 0

    def bfs(start):
        s = [[start, dict(), set()]]
        while s:
            i, color_dic, visited = s.pop()
            visited.add(i)
            color_dic[colors[i]] = color_dic.get(colors[i], 0) + 1
            nonlocal largest_color_value
            largest_color_value = max(largest_color_value, color_dic[colors[i]])
            if i in graph:
                for n in graph[i]:
                    if n in visited:
                        return False
                    else:
                        s.append([n, color_dic.copy(), visited.copy()])
        return True

    for node in from_nodes:
        if not bfs(node):
            return -1

    return largest_color_value

print(largestPathValue(colors = "abaca", edges = [[0,1],[0,2],[2,3],[3,4]])) # 3
print(largestPathValue(colors = "abaca", edges = [[0,1],[0,2],[2,3],[3,4],[4,1]])) # 3
print(largestPathValue(colors = "abaca", edges = [[0,1],[0,2],[2,3],[3,4],[4,2]])) # -1
print(largestPathValue(colors = "bbbhb", edges = [[0,2],[3,0],[1,3],[4,1]])) # 4
print(largestPathValue(colors = "a", edges = [[0,0]])) # -1
print(largestPathValue(colors = "g", edges = [])) # -1

3
3
-1
4
-1
1


In [72]:
# uses topological sort - very fast!
def largestPathValue(colors: str, edges: list[list[int]]) -> int:

    # no edges = max path is 1
    if not edges:
        return 1
    
    digraph = {} # storing the graph
    indegrees = {} # storing number of indegrees

    for k, v in edges:
        if k in digraph:
            digraph[k].append(v) # appending an edge
        else:
            digraph[k] = [v] # adding an edge to a node without any edges
        if v not in digraph:
            # if an edge leads to a node without any outcoming edges, we add that node to the dictionary
            # this step is needed to store nodes with no outcoming edges
            digraph[v] = []
        # counting indegrees
        indegrees[v] = indegrees.get(v, 0) + 1

    topological_sort = []
    # digraph contains all nodes as keys. indegrees contains only the nodes with incoming edges
    no_incoming_edges = list(digraph.keys() - indegrees.keys()) 

    # populating topological sort
    while no_incoming_edges:
        node = no_incoming_edges.pop()
        topological_sort.append(node)
        # iterating through the outcoming edges of the popped node
        for n in digraph[node]:
            # decrementing their indegrees
            indegrees[n] -= 1
            # if the popped node was their last remaining incoming edge, we add them to the topological sort
            if indegrees[n] == 0:
                no_incoming_edges.append(n)

    # this means there is a loop
    if len(digraph) != len(topological_sort):
        return -1
    
    largest_color_value = 0

    # maintaining a color dictionary for each node
    colour_dicts = {k:dict() for k in digraph.keys()}

    # going backwards in the topological sort
    for i in range(len(topological_sort)-1, -1, -1):
        index = topological_sort[i]

        # iterating through outgoing edges
        for n in digraph[index]:
            for color in colour_dicts[n]:
                # choosing maximum possible number of colours available to any child
                colour_dicts[index][color] = max(colour_dicts[index].get(color, 0), colour_dicts[n][color])
        # incrementing the node colour counter
        colour_dicts[index][colors[index]] = colour_dicts[index].get(colors[index], 0) + 1
        # potentially updating the largest_color_value counter
        largest_color_value = max(largest_color_value, colour_dicts[index][colors[index]])

    return largest_color_value


print(largestPathValue(colors = "abaca", edges = [[0,1],[0,2],[2,3],[3,4]])) # 3
print(largestPathValue(colors = "abaca", edges = [[0,1],[0,2],[2,3],[3,4],[4,1]])) # 3
print(largestPathValue(colors = "abaca", edges = [[0,1],[0,2],[2,3],[3,4],[4,2]])) # -1
print(largestPathValue(colors = "bbbhb", edges = [[0,2],[3,0],[1,3],[4,1]])) # 4
print(largestPathValue(colors = "a", edges = [[0,0]])) # -1
print(largestPathValue(colors = "g", edges = [])) # -1

3
3
-1
4
-1
1


**2390. Removing Stars From a String**

You are given a string s, which contains stars *.

In one operation, you can:

Choose a star in s.
Remove the closest non-star character to its left, as well as remove the star itself.
Return the string after all stars have been removed.

Note:

The input will be generated such that the operation is always possible.
It can be shown that the resulting string will always be unique.

https://leetcode.com/problems/removing-stars-from-a-string/

In [65]:
def removeStars(s: str) -> str:
    stack = []
    for c in s:
        if c == "*":
            stack.pop()
        else:
            stack.append(c)
    return "".join(stack)

print(removeStars(s = "leet**cod*e")) # lecoe

lecoe
